# Download Radiosonde Data and Calculate Parameters Significant to Tornadogensis.

In [2]:
import siphon
from siphon.simplewebservice.wyoming import WyomingUpperAir

import metpy
import matplotlib.pyplot as plt
from metpy.plots import SkewT
import metpy.calc as calcs
from metpy.units import units

import numpy as np
import pandas as pd

from datetime import datetime
import time

import warnings
warnings.filterwarnings("ignore", category=UserWarning)

## List of Stations with Radiosonde Data
Here is a list of all the staions with data. This includes some stations in Canada to get a complete raster of the CONUS. Data is pulled from https://weather.uwyo.edu/upperair/sounding.html

In [3]:
hours = [0, 12]
Stations = ['UIL', 'SLE', 'MFR', 'OAK', 'NKX', 'OTX', 'BOI', 'REV', 'LKN', 'VEF', 'TFX', 'SLC', 'FGZ', 'TUS', 'GGW', 'ABQ', 'EPZ', 'DNR',
            'BIS', 'ABR', 'OAX', 'LBF', 'DDC', 'TOP', 'OUN', 'AMA', 'MAF', 'FWD', 'DRT', 'CRP', 'BRO', 'INL', 'MPX', 'DVN', 'SGF', 'LZK',
            'SHV', 'LCH', 'LIX', 'JAN', 'ILX', 'GRB', 'APX', 'DTX', 'ILN', 'BMX', 'FFC', 'TBW', 'EYW', 'MFL', 'JAX', 'CHS', 'MHX',
            'RNK', 'WAL', 'BUF', 'WAL', 'OKX', 'CHH', 'GYX', 'CAR', 'TLH', 'IAQ', 'ALB', 'BUF', 'AWE', 'YQI', 'WMW', 'WPL', 'VQD', 'WSE', 'YZT']

## Data Creation Loops

In [4]:
before = datetime.now()

#Open the text file that values will be printed to.
file_object = open('sonde_data_dec_2019.txt', 'a')

#Interate through the list of radiosonde stations
for station in Stations:
    
    #Reset/preset the lists
    row=[]
    srh_tot=[]
    cape_tot=[]
    dew_tot=[]
    
    #Interate through the day and hours of radiosonde luanches
    for day in range(30):
        for hour in hours:
        
            #Use a try statement to keep the program running if data or the server isn't availble
            try:
                
                #Request the data
                request_time = datetime(2019, 12, day+1, hour)
                df = WyomingUpperAir.request_data(request_time, station)

                #grab different values
                u = df.u_wind.values * units.knot
                v = df.v_wind.values * units.knot
                h = df.height.values * units.meter
                p = df.pressure.values * units.hPa
                d = df.dewpoint.values * units.celsius
                t = df.temperature.values * units.celsius
                lat = df.latitude.values
                lon = df.longitude.values
                
                #Calculate tornado parameters using MetPy and append values
                cape, cin = calcs.surface_based_cape_cin(p, t, d)
                srh_1_pos, srh_1_neg, srh_1_tot = calcs.storm_relative_helicity(h, u, v, 1000 * units.meter)
                srh_tot.append(srh_1_tot.m)
                cape_tot.append(cape.m)
                dew_tot.append(d.m[0])

            #Skip any timeslots that don't have data
            #If the server is slow, the script will sleep and then restart a minute later
            except ValueError:
                print(f"there is no data for {day+1} at {hour} at {station}.")
                pass

            except:
                print(f"Waiting for this slow ass server to hurry up. {station} is being skipped")
                time.sleep(60)
                pass
        
    row.append((f'{station}, {np.average(srh_tot)}, {np.average(cape_tot)}, {np.average(dew_tot)}, {sum(shear >= 100 for shear in srh_tot)}, {sum(capes >= 1000 for capes in cape_tot)}, {sum(dews >= 12.8 for dews in dew_tot)}, {lat[0]}, {lon[0]} \n’'))
    print(row)

    #write the values calculated to the text file
    file_object.write(f'{station}, {np.average(srh_tot)}, {np.average(cape_tot)}, {np.average(dew_tot)}, {sum(shear >= 100 for shear in srh_tot)}, {sum(capes >= 1000 for capes in cape_tot)}, {sum(dews >= 12.8 for dews in dew_tot)}, {lat[0]}, {lon[0]} \n')
    
#close the text file    
file_object.close()
after = datetime.now()
print(after - before)

['UIL, 70.74142737981248, 10.302740817130685, 4.976666666666666, 16, 0, 0, 47.95, -124.55 \n’']
['SLE, 34.720521156560444, 10.376863252165888, 4.648333333333334, 6, 0, 0, 44.91, -123.0 \n’']
['MFR, 21.52409440160663, 16.908814527771366, 2.9366666666666665, 1, 0, 0, 42.36, -122.86 \n’']
there is no data for 15 at 0 at OAK.
there is no data for 15 at 12 at OAK.
there is no data for 16 at 0 at OAK.
there is no data for 16 at 12 at OAK.
['OAK, 43.885101252676, 13.159602687036962, 8.489285714285714, 8, 0, 5, 37.73, -122.21 \n’']
Waiting for this slow ass server to hurry up. NKX is being skipped
Waiting for this slow ass server to hurry up. NKX is being skipped
Waiting for this slow ass server to hurry up. NKX is being skipped
['NKX, 4.2284503355206455, 58.650491085771506, 8.945614035087718, 0, 0, 14, 32.85, -117.12 \n’']
Waiting for this slow ass server to hurry up. OTX is being skipped
Waiting for this slow ass server to hurry up. OTX is being skipped
['OTX, 50.96416863186374, 2.2096335968

/anaconda3/envs/pyEAE/lib/python3.9/site-packages/metpy/interpolate/one_dimension.py:157: RuntimeWarning: invalid value encountered in true_divide
  var_interp = var[below] + (var[above] - var[below]) * ((x_array - xp[below])


['GRB, 72.5612072987232, 0.2938190724703135, -5.508333333333334, 19, 0, 0, 44.5, -88.11 \n’']
['APX, 79.01678029630125, 2.9719551511799387, -5.043333333333334, 19, 0, 0, 44.91, -84.71 \n’']
there is no data for 2 at 0 at DTX.
there is no data for 2 at 12 at DTX.
there is no data for 3 at 0 at DTX.
there is no data for 3 at 12 at DTX.
there is no data for 24 at 12 at DTX.
['DTX, 93.96550505731483, 1.5817110377414776, -2.878181818181818, 22, 0, 0, 42.7, -83.46 \n’']
['ILN, 69.63780809680864, 3.737962057592541, 0.2016666666666667, 17, 0, 1, 39.41, -83.81 \n’']
['BMX, 66.00387649400537, 20.50170690313696, 6.4700000000000015, 23, 0, 12, 33.16, -86.76 \n’']
Waiting for this slow ass server to hurry up. FFC is being skipped
['FFC, 53.51601749790479, 1.7761618311124705, 5.569491525423728, 11, 0, 11, 33.36, -84.57 \n’']
there is no data for 13 at 0 at TBW.
Waiting for this slow ass server to hurry up. TBW is being skipped
['TBW, 44.779870168232044, 239.52174898572287, 15.879310344827585, 16, 5,

/anaconda3/envs/pyEAE/lib/python3.9/site-packages/numpy/lib/function_base.py:495: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/anaconda3/envs/pyEAE/lib/python3.9/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


['ALB, 67.60105093658852, 1.8213086784171337, -5.138333333333334, 16, 0, 0, 42.69, -73.83 \n’']
['BUF, 87.10591149889107, 1.5735979315211213, -3.0316666666666663, 20, 0, 0, 42.93, -78.73 \n’']
there is no data for 1 at 0 at AWE.
there is no data for 1 at 12 at AWE.
there is no data for 2 at 0 at AWE.
there is no data for 2 at 12 at AWE.
there is no data for 3 at 0 at AWE.
there is no data for 3 at 12 at AWE.
there is no data for 4 at 0 at AWE.
there is no data for 4 at 12 at AWE.
there is no data for 5 at 0 at AWE.
there is no data for 5 at 12 at AWE.
there is no data for 6 at 0 at AWE.
there is no data for 6 at 12 at AWE.
there is no data for 7 at 0 at AWE.
there is no data for 7 at 12 at AWE.
there is no data for 8 at 0 at AWE.
there is no data for 8 at 12 at AWE.
there is no data for 9 at 0 at AWE.
there is no data for 9 at 12 at AWE.
there is no data for 10 at 0 at AWE.
there is no data for 10 at 12 at AWE.
there is no data for 11 at 0 at AWE.
there is no data for 11 at 12 at AWE.


/anaconda3/envs/pyEAE/lib/python3.9/site-packages/numpy/lib/function_base.py:495: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/anaconda3/envs/pyEAE/lib/python3.9/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


['YQI, 51.07996953894193, 0.811602364383074, -2.5733333333333333, 13, 0, 1, 43.87, -66.11 \n’']
['WMW, 75.54268700405177, 1.3049346535792086, -9.701666666666664, 20, 0, 0, 46.3, -76.01 \n’']
['WPL, 59.805650330883324, 0.04052261144302425, -17.241666666666667, 14, 0, 0, 51.45, -90.22 \n’']
there is no data for 1 at 0 at VQD.
there is no data for 1 at 12 at VQD.
there is no data for 2 at 0 at VQD.
there is no data for 2 at 12 at VQD.
there is no data for 3 at 0 at VQD.
there is no data for 3 at 12 at VQD.
there is no data for 4 at 0 at VQD.
there is no data for 4 at 12 at VQD.
there is no data for 5 at 0 at VQD.
there is no data for 5 at 12 at VQD.
there is no data for 6 at 0 at VQD.
there is no data for 6 at 12 at VQD.
there is no data for 7 at 0 at VQD.
there is no data for 7 at 12 at VQD.
there is no data for 8 at 0 at VQD.
there is no data for 8 at 12 at VQD.
there is no data for 9 at 0 at VQD.
there is no data for 9 at 12 at VQD.
there is no data for 10 at 0 at VQD.
there is no data

/anaconda3/envs/pyEAE/lib/python3.9/site-packages/numpy/lib/function_base.py:495: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/anaconda3/envs/pyEAE/lib/python3.9/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


['WSE, 29.458346136113718, 0.0022929338245683477, -11.255, 6, 0, 0, 53.53, -114.1 \n’']
['YZT, 30.58375357841212, 1.3306335542867171, 2.9066666666666667, 6, 0, 0, 50.68, -127.36 \n’']
2:44:20.637732
